# ACTIVITY 4: MELODY MATCH

#### This section performs topic modeling on a set of lemmatized song lyrics using the Latent Dirichlet Allocation (LDA) technique. It first tokenizes the lyrics into individual words and creates a dictionary mapping each word to a unique identifier. The tokenized lyrics are then converted into a bag-of-words (BoW) representation, which is used as input for the LDA model. The model is trained to identify 10 topics, running for 35 passes to ensure robust learning. Finally, it calculates the topic distribution for each song, showing the proportion of each identified topic in the corresponding lyrics.

#### The output of the code is displaying the songs most similar to the user's input using DataFrame showing the 6 most similar songs based on the calculated factors.

In [ ]:
import numpy as np
import pandas as pd
import nltk
import re
import spacy
from gensim import corpora, models
import gensim
from gensim.matutils import hellinger
from scipy.spatial.distance import cosine
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
df = pd.read_csv(r'FullyProcessedDataset.csv')

tokenized_corpus = [text.split() for text in df['Lemmatized_Lyrics']]

dictionary = corpora.Dictionary(tokenized_corpus)
corpus = [dictionary.doc2bow(text) for text in tokenized_corpus]

lda_model = gensim.models.LdaModel(corpus, num_topics=10, id2word=dictionary, passes=35, random_state = 42)

song_topic_distribution = [lda_model[doc] for doc in corpus]

In [ ]:

user_input = input("Enter song index: ")
user_song = (df.index[df['SName'] == user_input]).item()
print(user_song)
df.head(5)


Enter song index: 3
8490


,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,ALink,SName,SLink,Lyric,language,Lemmatized_Lyrics,Sentiment,Artist,Genres
0,0,0,139419,/foo-fighters/,"Hey, Johnny Park!",/foo-fighters/hey-johnny-park.html,Come and I'll take you under\nThis beautiful b...,en,come take beautiful bruise color everything fa...,0.8316,Foo Fighters,Rock Alternativo; Rock; Hard Rock
1,1,46,139365,/foo-fighters/,Baker Street,/foo-fighters/baker-street.html,Winding you way down on baker street\nLight in...,en,winding way baker street light head dead foot ...,-0.1561,Foo Fighters,Rock Alternativo; Rock; Hard Rock
2,2,315,139477,/foo-fighters/,Savior Breath,/foo-fighters/savior-breath.html,[Verse 1]\nBreathe oh breathe now smother me\n...,en,verse breathe oh breathe smother clever onto o...,0.0854,Foo Fighters,Rock Alternativo; Rock; Hard Rock
3,3,319,139496,/foo-fighters/,The Line,/foo-fighters/the-line.html,Yes or no?\nWhat is truth\nBut a dirty black c...,en,yes truth dirty black cloud come blue wrong ri...,-0.9337,Foo Fighters,Rock Alternativo; Rock; Hard Rock
4,4,835,139485,/foo-fighters/,Statues,/foo-fighters/statues.html,You and I were two old and tortured souls\nRep...,en,two old torture soul repair love break fang li...,0.0258,Foo Fighters,Rock Alternativo; Rock; Hard Rock


#### The piece of code computes the Hellinger distance using *calculate_hellinger_distance* between two probability distributions, which in this case, represent the topic distributions of two songs. The code first retrieves the topic distribution (input_song_dist) of the song selected by the user using its index (user_song). It then iterates over all song topic distributions in song_topic_distribution, calculating the Hellinger distance between the input song's distribution and each other song's distribution. These distances are appended to a list (hellinger_distances). Finally, this list is added to the DataFrame df as a new column, HDF, which represents the Hellinger distances for each song compared to the input song. The first 60 rows of the updated DataFrame are displayed to the user.

In [ ]:
def calculate_hellinger_distance(song_dist_1, song_dist_2):
    return hellinger(song_dist_1, song_dist_2)

input_song_index = user_song

input_song_dist = song_topic_distribution[input_song_index]

hellinger_distances = []

for i, song_dist in enumerate(song_topic_distribution):

    distance = calculate_hellinger_distance(input_song_dist, song_dist)
    hellinger_distances.append((distance))

df['HDF'] = hellinger_distances
df.head(60)

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,ALink,SName,SLink,Lyric,language,Lemmatized_Lyrics,Sentiment,Artist,Genres,HDF
0,0,0,139419,/foo-fighters/,"Hey, Johnny Park!",/foo-fighters/hey-johnny-park.html,Come and I'll take you under\nThis beautiful b...,en,come take beautiful bruise color everything fa...,0.8316,Foo Fighters,Rock Alternativo; Rock; Hard Rock,0.341428
1,1,46,139365,/foo-fighters/,Baker Street,/foo-fighters/baker-street.html,Winding you way down on baker street\nLight in...,en,winding way baker street light head dead foot ...,-0.1561,Foo Fighters,Rock Alternativo; Rock; Hard Rock,0.495982
2,2,315,139477,/foo-fighters/,Savior Breath,/foo-fighters/savior-breath.html,[Verse 1]\nBreathe oh breathe now smother me\n...,en,verse breathe oh breathe smother clever onto o...,0.0854,Foo Fighters,Rock Alternativo; Rock; Hard Rock,0.496834
3,3,319,139496,/foo-fighters/,The Line,/foo-fighters/the-line.html,Yes or no?\nWhat is truth\nBut a dirty black c...,en,yes truth dirty black cloud come blue wrong ri...,-0.9337,Foo Fighters,Rock Alternativo; Rock; Hard Rock,0.262788
4,4,835,139485,/foo-fighters/,Statues,/foo-fighters/statues.html,You and I were two old and tortured souls\nRep...,en,two old torture soul repair love break fang li...,0.0258,Foo Fighters,Rock Alternativo; Rock; Hard Rock,0.267466
5,5,1837,139500,/foo-fighters/,This Will Be Our Year,/foo-fighters/this-will-be-our-year.html,The warmth of your love's\nLike the warmth fro...,en,warmth love like warmth sun year take long tim...,0.9870,Foo Fighters,Rock Alternativo; Rock; Hard Rock,0.521665
6,6,2478,139457,/foo-fighters/,No Way Back,/foo-fighters/no-way-back.html,"Lately, I've been\nLivin' in my head\nThe rest...",en,lately livin head rest dead die truth make bel...,0.9834,Foo Fighters,Rock Alternativo; Rock; Hard Rock,0.667903
7,7,2800,139494,/foo-fighters/,The Feast and The Famine,/foo-fighters/the-feast-and-the-famine.html,That night they were burning of the truth\nDow...,en,night burn truth corner th u take soul take fo...,-0.8419,Foo Fighters,Rock Alternativo; Rock; Hard Rock,0.512640
8,8,4356,139407,/foo-fighters/,Gimme Stitches,/foo-fighters/gimme-stitches.html,I can be your right of way\nSo we can get out ...,en,right way get always one run everyone cause ev...,-0.8420,Foo Fighters,Rock Alternativo; Rock; Hard Rock,0.330715
9,9,5613,139454,/foo-fighters/,Never Talking To You Again,/foo-fighters/never-talking-to-you-again-2.html,There are things that I'd like to say\nBut I'm...,en,thing like say never talk thing like phrase wa...,0.6846,Foo Fighters,Rock Alternativo; Rock; Hard Rock,0.631537


In [ ]:
userGenre = df['Genres'].iloc[user_song]
genreReducedDf = df.loc[df['Genres'] == userGenre]
genreReducedDf.reset_index(inplace = True)
print(df.shape[0])
print(genreReducedDf.shape[0])
genreReducedDf.head(45)


27962
368


,index,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,ALink,SName,SLink,Lyric,language,Lemmatized_Lyrics,Sentiment,Artist,Genres,HDF
0,4844,4844,144,143181,/led-zeppelin/,Good Times Bad Times,/led-zeppelin/good-times-bad-times.html,"In the days of my youth, I was told what it me...",en,day youth tell mean man reach age try thing we...,0.9899,Led Zeppelin,Rock; Hard Rock; Heavy Metal,0.625179
1,4845,4845,1163,143221,/led-zeppelin/,Immigrant Song (Dirty Funker Remix),/led-zeppelin/immigrant-song-dirty-funker-remi...,"Ah, Ah, Ah, Ah\nWe come from the land of the i...",en,ah ah ah ah come land ice snow midnight sun ho...,0.4133,Led Zeppelin,Rock; Hard Rock; Heavy Metal,0.636360
2,4846,4846,4948,143173,/led-zeppelin/,D'yer Mak'er,/led-zeppelin/dyer-maker.html,Oh oh oh oh oh oh\nYou don't have to go oh oh ...,en,oh oh oh oh oh oh go oh oh oh oh oh go oh oh o...,-0.8910,Led Zeppelin,Rock; Hard Rock; Heavy Metal,0.820472
3,4847,4847,8259,143199,/led-zeppelin/,Candy Store Rock,/led-zeppelin/candy-store-rock.html,(Page/Plant)\n\nWell... Oh baby baby. Don't yo...,en,pageplant well oh baby baby want man like oh b...,0.9940,Led Zeppelin,Rock; Hard Rock; Heavy Metal,0.763132
4,4848,4848,15242,143237,/led-zeppelin/,Somethin' Else,/led-zeppelin/somethin-else.html,"Oh, look over there\nHere she comes\nHere come...",en,oh look come come girl one know since do not k...,0.9605,Led Zeppelin,Rock; Hard Rock; Heavy Metal,0.585316
5,4849,4849,16328,143182,/led-zeppelin/,You Shook Me,/led-zeppelin/you-shook-me.html,You know you shook me\nYou shook me all night ...,en,know shake shake night long know shake baby sh...,-0.8316,Led Zeppelin,Rock; Hard Rock; Heavy Metal,0.923624
6,4850,4850,16885,143170,/led-zeppelin/,Kashmir,/led-zeppelin/kashmir.html,"Oh let the sun beat down upon my face,\nstars ...",en,oh let sun beat upon face star fill dream trav...,0.9099,Led Zeppelin,Rock; Hard Rock; Heavy Metal,0.385832
7,4851,4851,17540,143202,/led-zeppelin/,Communication Breakdown,/led-zeppelin/communication-breakdown.html,"Hey, girl, stop what you're doin'!\nHey, girl,...",en,hey girl stop doin hey girl drive ruin know li...,0.8271,Led Zeppelin,Rock; Hard Rock; Heavy Metal,0.837419
8,4852,4852,25501,143193,/led-zeppelin/,Black Country Woman,/led-zeppelin/black-country-woman.html,"Hey, hey, mama, what's the matter here (X2)\nY...",en,hey hey mama matter x tell love love mama let ...,0.9493,Led Zeppelin,Rock; Hard Rock; Heavy Metal,0.762697
9,4853,4853,32876,143234,/led-zeppelin/,Rain Song,/led-zeppelin/rain-song.html,This is the springtime of my loving - the seco...,en,springtime love second season know sunlight gr...,0.8669,Led Zeppelin,Rock; Hard Rock; Heavy Metal,0.289389


#### This below code allows the user to input a song name, retrieves its index, and calculates its sentiment score using the VADER sentiment analyzer. It then computes the Sentiment Similarity Factor (SSF) for each song in the DataFrame by calculating the absolute difference between the input song's sentiment score and every other song's sentiment score. These SSF values are stored in a new column, SSF, and the first 56 rows of the updated DataFrame are displayed. The SSF indicates how similar each song's sentiment is to the input song's sentiment.

In [ ]:
userSongIndexUpdated = (genreReducedDf.index[genreReducedDf['SName'] == user_input]).item()
print(userSongIndexUpdated)
analyzer = SentimentIntensityAnalyzer()

def get_sentiment_score(lyrics):
    sentiment = analyzer.polarity_scores(lyrics)
    sentiment_score = sentiment['compound']
    return sentiment_score

sentiment = get_sentiment_score(genreReducedDf['Lemmatized_Lyrics'].iloc[userSongIndexUpdated])
print(sentiment)

SSF_list = []
for i in range(genreReducedDf.shape[0]):
    SSF_list.append(abs(genreReducedDf['Sentiment'].iloc[i] - sentiment))

genreReducedDf['SSF'] = SSF_list
genreReducedDf.head(56)



109
-0.9977


<ipython-input-7-d0763adcb25f>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genreReducedDf['SSF'] = SSF_list


,index,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,ALink,SName,SLink,Lyric,language,Lemmatized_Lyrics,Sentiment,Artist,Genres,HDF,SSF
0,4844,4844,144,143181,/led-zeppelin/,Good Times Bad Times,/led-zeppelin/good-times-bad-times.html,"In the days of my youth, I was told what it me...",en,day youth tell mean man reach age try thing we...,0.9899,Led Zeppelin,Rock; Hard Rock; Heavy Metal,0.625179,1.9876
1,4845,4845,1163,143221,/led-zeppelin/,Immigrant Song (Dirty Funker Remix),/led-zeppelin/immigrant-song-dirty-funker-remi...,"Ah, Ah, Ah, Ah\nWe come from the land of the i...",en,ah ah ah ah come land ice snow midnight sun ho...,0.4133,Led Zeppelin,Rock; Hard Rock; Heavy Metal,0.636360,1.4110
2,4846,4846,4948,143173,/led-zeppelin/,D'yer Mak'er,/led-zeppelin/dyer-maker.html,Oh oh oh oh oh oh\nYou don't have to go oh oh ...,en,oh oh oh oh oh oh go oh oh oh oh oh go oh oh o...,-0.8910,Led Zeppelin,Rock; Hard Rock; Heavy Metal,0.820472,0.1067
3,4847,4847,8259,143199,/led-zeppelin/,Candy Store Rock,/led-zeppelin/candy-store-rock.html,(Page/Plant)\n\nWell... Oh baby baby. Don't yo...,en,pageplant well oh baby baby want man like oh b...,0.9940,Led Zeppelin,Rock; Hard Rock; Heavy Metal,0.763132,1.9917
4,4848,4848,15242,143237,/led-zeppelin/,Somethin' Else,/led-zeppelin/somethin-else.html,"Oh, look over there\nHere she comes\nHere come...",en,oh look come come girl one know since do not k...,0.9605,Led Zeppelin,Rock; Hard Rock; Heavy Metal,0.585316,1.9582
5,4849,4849,16328,143182,/led-zeppelin/,You Shook Me,/led-zeppelin/you-shook-me.html,You know you shook me\nYou shook me all night ...,en,know shake shake night long know shake baby sh...,-0.8316,Led Zeppelin,Rock; Hard Rock; Heavy Metal,0.923624,0.1661
6,4850,4850,16885,143170,/led-zeppelin/,Kashmir,/led-zeppelin/kashmir.html,"Oh let the sun beat down upon my face,\nstars ...",en,oh let sun beat upon face star fill dream trav...,0.9099,Led Zeppelin,Rock; Hard Rock; Heavy Metal,0.385832,1.9076
7,4851,4851,17540,143202,/led-zeppelin/,Communication Breakdown,/led-zeppelin/communication-breakdown.html,"Hey, girl, stop what you're doin'!\nHey, girl,...",en,hey girl stop doin hey girl drive ruin know li...,0.8271,Led Zeppelin,Rock; Hard Rock; Heavy Metal,0.837419,1.8248
8,4852,4852,25501,143193,/led-zeppelin/,Black Country Woman,/led-zeppelin/black-country-woman.html,"Hey, hey, mama, what's the matter here (X2)\nY...",en,hey hey mama matter x tell love love mama let ...,0.9493,Led Zeppelin,Rock; Hard Rock; Heavy Metal,0.762697,1.9470
9,4853,4853,32876,143234,/led-zeppelin/,Rain Song,/led-zeppelin/rain-song.html,This is the springtime of my loving - the seco...,en,springtime love second season know sunlight gr...,0.8669,Led Zeppelin,Rock; Hard Rock; Heavy Metal,0.289389,1.8646


#### This below code calculates the Cosine Similarity Factor (CSF) between the song selected by the user and all other songs in the DataFrame based on their lemmatized lyrics. It uses a TF-IDF vectorizer to transform the lyrics into numerical representations and computes the cosine similarity between the selected song and each other song. The CSF is derived by subtracting the cosine similarity from 1, and this value is stored in a new column CSF in the DataFrame. The first 30 rows of the updated DataFrame are then displayed. The CSF represents how similar each song is to the input song in terms of lyrics.

In [ ]:
tfidf_vectorizer = TfidfVectorizer()
CSF_list = []
for i in range(genreReducedDf.shape[0]):
    lyrics_matrix = tfidf_vectorizer.fit_transform([genreReducedDf['Lemmatized_Lyrics'].iloc[userSongIndexUpdated], genreReducedDf['Lemmatized_Lyrics'].iloc[i]])
    lyrics_similarity = cosine_similarity(lyrics_matrix)
    CSF_list.append(1 - (lyrics_similarity[0][1]))
genreReducedDf['CSF'] = CSF_list
genreReducedDf.head(30)


<ipython-input-8-0bf795056c54>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genreReducedDf['CSF'] = CSF_list


,index,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,ALink,SName,SLink,Lyric,language,Lemmatized_Lyrics,Sentiment,Artist,Genres,HDF,SSF,CSF
0,4844,4844,144,143181,/led-zeppelin/,Good Times Bad Times,/led-zeppelin/good-times-bad-times.html,"In the days of my youth, I was told what it me...",en,day youth tell mean man reach age try thing we...,0.9899,Led Zeppelin,Rock; Hard Rock; Heavy Metal,0.625179,1.9876,0.955379
1,4845,4845,1163,143221,/led-zeppelin/,Immigrant Song (Dirty Funker Remix),/led-zeppelin/immigrant-song-dirty-funker-remi...,"Ah, Ah, Ah, Ah\nWe come from the land of the i...",en,ah ah ah ah come land ice snow midnight sun ho...,0.4133,Led Zeppelin,Rock; Hard Rock; Heavy Metal,0.636360,1.4110,0.994689
2,4846,4846,4948,143173,/led-zeppelin/,D'yer Mak'er,/led-zeppelin/dyer-maker.html,Oh oh oh oh oh oh\nYou don't have to go oh oh ...,en,oh oh oh oh oh oh go oh oh oh oh oh go oh oh o...,-0.8910,Led Zeppelin,Rock; Hard Rock; Heavy Metal,0.820472,0.1067,0.992842
3,4847,4847,8259,143199,/led-zeppelin/,Candy Store Rock,/led-zeppelin/candy-store-rock.html,(Page/Plant)\n\nWell... Oh baby baby. Don't yo...,en,pageplant well oh baby baby want man like oh b...,0.9940,Led Zeppelin,Rock; Hard Rock; Heavy Metal,0.763132,1.9917,0.990730
4,4848,4848,15242,143237,/led-zeppelin/,Somethin' Else,/led-zeppelin/somethin-else.html,"Oh, look over there\nHere she comes\nHere come...",en,oh look come come girl one know since do not k...,0.9605,Led Zeppelin,Rock; Hard Rock; Heavy Metal,0.585316,1.9582,0.984445
5,4849,4849,16328,143182,/led-zeppelin/,You Shook Me,/led-zeppelin/you-shook-me.html,You know you shook me\nYou shook me all night ...,en,know shake shake night long know shake baby sh...,-0.8316,Led Zeppelin,Rock; Hard Rock; Heavy Metal,0.923624,0.1661,1.000000
6,4850,4850,16885,143170,/led-zeppelin/,Kashmir,/led-zeppelin/kashmir.html,"Oh let the sun beat down upon my face,\nstars ...",en,oh let sun beat upon face star fill dream trav...,0.9099,Led Zeppelin,Rock; Hard Rock; Heavy Metal,0.385832,1.9076,0.934228
7,4851,4851,17540,143202,/led-zeppelin/,Communication Breakdown,/led-zeppelin/communication-breakdown.html,"Hey, girl, stop what you're doin'!\nHey, girl,...",en,hey girl stop doin hey girl drive ruin know li...,0.8271,Led Zeppelin,Rock; Hard Rock; Heavy Metal,0.837419,1.8248,0.987668
8,4852,4852,25501,143193,/led-zeppelin/,Black Country Woman,/led-zeppelin/black-country-woman.html,"Hey, hey, mama, what's the matter here (X2)\nY...",en,hey hey mama matter x tell love love mama let ...,0.9493,Led Zeppelin,Rock; Hard Rock; Heavy Metal,0.762697,1.9470,0.979461
9,4853,4853,32876,143234,/led-zeppelin/,Rain Song,/led-zeppelin/rain-song.html,This is the springtime of my loving - the seco...,en,springtime love second season know sunlight gr...,0.8669,Led Zeppelin,Rock; Hard Rock; Heavy Metal,0.289389,1.8646,0.933185


#### This code calculates a combined Similarity Factor (SF) for each song by summing the SSF, CSF, and HDF values. It then selects the top 6 songs with the smallest SF values, indicating the songs most similar to the user's input. The song corresponding to the user's input is removed from the DataFrame. Finally, the updated DataFrame is displayed, showing the 6 most similar songs based on the calculated factors.

In [ ]:
genreReducedDf['SF'] = genreReducedDf['SSF'] + genreReducedDf['CSF'] + genreReducedDf['HDF']
genreReducedDf = genreReducedDf.nsmallest(6,["SF"])
genreReducedDf.drop(userSongIndexUpdated, axis = 0, inplace = True)
genreReducedDf

<ipython-input-9-99c83b81700b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genreReducedDf['SF'] = genreReducedDf['SSF'] + genreReducedDf['CSF'] + genreReducedDf['HDF']


,index,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,ALink,SName,SLink,Lyric,language,Lemmatized_Lyrics,Sentiment,Artist,Genres,HDF,SSF,CSF,SF
143,11813,11813,1771,142446,/metallica/,Ride The Lightning,/metallica/ride-the-lightning.html,"Guilty as charged\nBut damn it, it ain't right...",en,guilty charge damn right someone else control ...,-0.9666,Metallica,Rock; Hard Rock; Heavy Metal,0.282448,0.0311,0.854069,1.167617
119,10821,10821,407,151565,/dr-sin/,Behind Enemy Lies,/dr-sin/behind-enemy-lies.html,"---""There's one thing about life you got to kn...",en,one thing life got know learn live learn die f...,-0.9934,Dr. Sin,Rock; Hard Rock; Heavy Metal,0.242074,0.0043,0.936325,1.182698
173,12129,12129,14513,146945,/slash/,Beneath The Savage Sun,/slash/beneath-the-savage-sun.html,"(Hey, ah)\n\nIt was a blood red summer\nOne I ...",en,hey ah blood red summer one forget take one th...,-0.9684,Slash,Rock; Hard Rock; Heavy Metal,0.277339,0.0293,0.877155,1.183793
111,8492,8492,25217,145014,/disturbed/,The Infection,/disturbed/the-infection.html,In the back of my mind I can\nBarely even reme...,en,back mind barely even remember nothing leave h...,-0.9885,Disturbed,Rock; Hard Rock; Heavy Metal,0.250148,0.0092,0.933615,1.192963
101,8482,8482,6619,145000,/disturbed/,Serpentine,/disturbed/serpentine.html,See the dogs come running\nSmelling blood now\...,en,see dog come run smell blood open sore parasit...,-0.9771,Disturbed,Rock; Hard Rock; Heavy Metal,0.277050,0.0206,0.910818,1.208468


In [ ]:
RecommendationDF = genreReducedDf[['Artist', 'SName']]
RecommendationDF.reset_index(inplace = True)
RecommendationDF = RecommendationDF.drop('index', axis = 1)
RecommendationDF

,Artist,SName
0,Metallica,Ride The Lightning
1,Dr. Sin,Behind Enemy Lies
2,Slash,Beneath The Savage Sun
3,Disturbed,The Infection
4,Disturbed,Serpentine
